# AI Agents

Up until now we have built some durable GenAI applications that:

- Called LLMs and APIs
- Allowed for human interaction
- Looped

But we - the developer - encoded flow decisions into those apps. What if we wanted to make it more complex? What if we wanted our research application to:

- **Decide if it needs more information** after an initial web search
- **Execute multiple searches** based on what it learns
- **Analyze results** and determine if they're sufficient

This is where the **agentic loop** becomes essential - where your AI application makes autonomous decisions, executes actions, observes results, and repeats until it achieves its goal. In this section, we'll explore what makes an AI system "agentic" and why building these systems reliably is more challenging than simple chains.

### What Makes a GenAI Application Agentic?

When we give **agency** to the **LLM** to drive the flow of the application!

### Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/cdavisafc/non-deterministic-ai-agents`. The instructions will also be in the README.
2. Run the Worker: `uv run python -m worker`
3. Start the agent: `uv run python -m start_workflow`
4. Emphasize that this demo showcases the core principle of agentic applications: we give **agency** to the **LLM** to drive the flow of the application. Unlike traditional applications where developers hardcode every decision and branching path, this agent makes its own choices at runtime. The LLM decides which actions to take, when to gather more information, and when the goal has been achieved—all autonomously.
-->

### Let's talk a bit about Models (and APIs)

- **The first models** were designed to complete the prompt that was passed in.
    - The original OpenAI API was the **Completions API** - you provided text, and it continued that text
    - Example: You give it "The sky is", it returns "blue" or "filled with clouds"
- Since then, models have been created/optimize to respond in different ways.
    - And in March 2025, OpenAI released the **responses API**
    - Specifically designed for **agentic workflows**
    - Optimized for agents that need to make autonomous decisions and execute tools iteratively

In [ ]:
### The Responses API

resp = await client.responses.create(
    model="",  # Choose a model designed for agentic tasks
               # Models like gpt-4o, gpt-4o-mini are optimized for tool calling
    
    instructions="You are a helpful weather assistant. Provide clear, concise weather information.",
    # For agents, this describes the role/behavior of the agent
    # Defines HOW the agent should behave and make decisions
    
    input=[
        {"role": "user", "content": "What's the weather in California?"}
    ],
    # Gathers the context as the agent progresses (through iterations)
    # As the agent loops, this grows to include:
    #   - User messages
    #   - Agent responses
    #   - Tool call requests
    #   - Tool call results
    
    tools=[
        {
            "type": "function",
            "name": "get_weather_alerts",
            "description": "Get current weather alerts for a US state",
            "parameters": {...}
        }
    ],
    # A list of the APIs available for use by the agent
    # The agent will autonomously decide WHEN and IF to use these tools
    
    timeout=30,  # Maximum time to wait for the API response
)

### What is a Tool?

A **tool** is an external function that AI systems can call to perform specific tasks beyond just generating text. Tools take real actions in the world—they might:
- Query databases
- Call external APIs
- Search for information
- Perform calculations
- Book reservations
- Send emails
- Generate files

### How Agents Use Tools

Agents are **aware of the tools** they have available while attempting to achieve their goal. The agent:
1. Evaluates which tools are available
2. Decides which tool will help progress toward the goal
3. Executes the tool
4. Observes the result
5. Decides the next action based on what it learned

### How Agents Execute Tools

Each iteration of the agent follows the pattern:

1. **LLM decides what to do** → "I need to check the order status"
2. **Agent prepares the tool call** → Formats the request: `check_order(order_id="12345")`
3. **Tool executes** → Queries the database and returns: `{status: "shipped", tracking: "1Z999..."}`
4. **Agent updates context** → Adds the result to the conversation history
5. **Loop continues** → LLM sees the new information and decides the next action

This continues until the LLM determines the goal is achieved.

In [ ]:
### Tool Example

# Tools are defined as JSON schemas that tell the LLM:
# 1. What the tool does (description)
# 2. What inputs it needs (parameters)
# 3. Which inputs are required vs optional

{
    "tools": [
        {
            "name": "get_weather_alerts",
            # Unique identifier - this is how the LLM refers to the tool
            
            "description": "Retrieves current weather alerts for a given U.S. state",
            # Clear description helps the LLM decide WHEN to use this tool
            # Be specific! "Get weather" vs "Get weather ALERTS" changes behavior
            
            "parameters": {
                "type": "object",
                "properties": {
                    "state": {
                        "type": "string",
                        "description": "The U.S. state to get weather alerts for, e.g. 'California' or 'TX'."
                        # Examples in the description help the LLM format inputs correctly
                    }
                },
                "required": ["state"]  # List of required parameters
            }
        },
        {
            "name": "generate_random_number",
            "description": "Generates a random number with no input parameters",            
            "parameters": {
                "type": "object",
                "properties": {}  # Empty properties = no parameters needed
            }
        }
    ]
}

# Your application then:
# 1. Executes the actual function
# 2. Returns the result to the LLM
# 3. The LLM uses that result to continue toward its goal

### Tools Continued

Ultimately, your agents need to manage a **registry of tools** that maps tool descriptions to actual API calls.

When the LLM returns:
```json
{
  "type": "function_call",
  "name": "get_weather_alerts",
  "call_id": "call_abc123",
  "arguments": "{\"state\": \"CA\"}"
}
```

Your application needs to:
1. **Look up** which actual function corresponds to `get_weather_alerts`
2. **Parse** the arguments string into the correct data structure
3. **Execute** the real function (API call, database query, etc.)
4. **Return** the result back to the LLM

### Tool Registry Pattern

```python
# Example tool registry
TOOL_REGISTRY = {
    "get_weather_alerts": get_weather_alerts_function,
    "search_database": search_database_function,
    "send_email": send_email_function,
}
```
We will cover this in a dedicated agent workshop where you'll build a robust tool registry system.

### Context Engineering

Context engineering is the practice of managing what information gets passed to the LLM at each iteration of the agentic loop. Unlike prompt engineering (crafting a single prompt), context engineering deals with an *evolving conversation* that accumulates over **multiple iterations**.

<img
src="https://i.postimg.cc/s27DCXNw/Screenshot-2025-10-08-at-10-50-13-AM.png"
width="300"/>

As agents loop, the context grows:

```python
# Iteration 1:
input = [
    {"role": "user", "content": "What's the weather in California?"}
]

# Iteration 2 (after tool call):
input = [
    {"role": "user", "content": "What's the weather in California?"},
    {"type": "function_call", "name": "get_weather_alerts", "arguments":
'{"state": "CA"}'},
    {"type": "function_call_output", "output": "{...weather data...}"}
]
```

<img src="https://i.postimg.cc/SQv0V4ZG/context-engineering-cont.png"
width="400"/>

### Let's Build an Agent!

Now it's time to put everything together! We are going to build a **weather alert agent**. 

First, let's set up your notebook. Run the following code blocks to install various packages and tools necessary to run this notebook

In [1]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio litellm python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /Users/azhou/Desktop/edu-ai-workshop-mcp/env/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

  # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

In [2]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [3]:
 # Running this will download the Temporal CLI

!curl -sSf https://temporal.download/cli.sh | sh

temporal: Downloading Temporal CLI latest
temporal: Temporal CLI installed at /Users/azhou/.temporalio/bin/temporal
temporal: For convenience, we recommend adding it to your PATH
temporal: If using bash, run echo export PATH="\$PATH:/Users/azhou/.temporalio/bin" >> ~/.bashrc


### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

## Building Our Agent: Step by Step

Now we'll build our weather alert agent. Here's what it will do:

1. Takes a user query (like "What's the weather in California?")
2. Decides autonomously whether it needs to fetch weather data:
    - If the query requires weather alert information for a U.S. state, it calls the `get_weather_alerts` tool
    - If no tools are needed, it responds in haikus (per the system instructions)
3. Fetches current weather alerts from the [National Weather Service API](https://www.weather.gov/documentation/services-web-api) for a specific U.S. state when needed
4. Returns results in a readable format - taking the raw API data and presenting it clearly to the user

Let's start by defining the data structure we'll use to communicate with the OpenAI Responses API.

### Step 1 - Create the Request Dataclass

First, we'll create a dataclass that packages all the parameters needed for our LLM calls. This groups together the model name, agent instructions, conversation input, and available tools in one structure. 

In [ ]:
# TODO: Run this code block to load it into the program
from dataclasses import dataclass
from typing import Any

@dataclass
class OpenAIResponsesRequest:
    model: str
    instructions: str
    input: object
    tools: list[dict[str, Any]]

### Step 2 - Create the Activity for LLM Invocations

We will create an Activity which invokes the OpenAI LLM, taking in our `OpenAIResponsesRequest` dataclass.

It's a reusable function that handles all communication with OpenAI's LLM throughout our agent's execution.

_Why make this an Activity?_ If the LLM fails (e.g., Network timeout, rate limitting), Temporal records the response in its event history. If
your workflow crashes after getting the LLM response, on replay Temporal returns the same stored response instead of calling the LLM again (saving time and money!).


In [ ]:
# Step 1: Supply the `OpenAIResponsesRequest` we just loaded in as the `request` for the parameter
# Step 2: Run this code block to load it into the program
from temporalio import activity
from openai import AsyncOpenAI
from openai.types.responses import Response

@activity.defn
async def create(request: ) -> Response: # TODO: Supply the `OpenAIResponsesRequest` as the `request` for the parameter
    # Temporal best practice: Disable retry logic in OpenAI API client library.
    client = AsyncOpenAI(max_retries=0)

    resp = await client.responses.create(
        model=request.model,
        instructions=request.instructions,
        input=request.input,
        tools=request.tools,
        timeout=30,
    )

    return resp

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "create_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Understanding How the Request Dataclass Evolves

The `OpenAIResponsesRequest dataclass` isn't just used once - it's used multiple  times throughout your agent's execution, and the data inside it evolves. 

Example: "What's the weather in California?"

First LLM Call - Initial Query

When your agent starts, you make the first call with minimal context:

#### First LLM call - asking "What's the weather in California?"
```python
result = await workflow.execute_activity(
    create,
    OpenAIResponsesRequest(
        model="gpt-4o-mini",
        instructions="if no tools seem to be needed, respond in haikus.",
        input=[{"role": "user", "content": "What's the weather in California?"}],
        tools=[WEATHER_ALERTS_TOOL_OAI]  # Available tool: get_weather_alerts
    ),
    start_to_close_timeout=timedelta(seconds=30),
)
```

The LLM decides it needs the weather alerts tool to answer this question.

Second LLM Call - With Tool Results

Now you make a second call, but notice how the input field has grown:

#### Second LLM call - with tool results included
```python
result = await workflow.execute_activity(
    create,
    OpenAIResponsesRequest(
        model="gpt-4o-mini",
        instructions="return the tool call result in a readable format",
        input=[
            {"role": "user", "content": "What's the weather in California?"},
            {"type": "function_call", "name": "get_weather_alerts", "call_id":
"call_abc123", "arguments": "{\"state\": \"CA\"}"},
            {"type": "function_call_output", "call_id": "call_abc123", "output":
"...actual weather data..."}
        ],
        tools=[]  # No more tools needed - we have the data!
    ),
    start_to_close_timeout=timedelta(seconds=30),
)
```
The LLM now has all the information it needs to provide a complete answer.

### Step 3 - Create the Helper Function

Before we build our tool that fetches weather alerts, we need a helper function that converts Pydantic models into OpenAI's tool definition format.

The `oai_responses_tool_from_model` function accepts a tool name, description, and a Pydantic model, then returns JSON in the format expected by OpenAI's Responses API tool definitions.

**Example output format:**
```json
{
  "type": "function",
  "name": "get_weather_alerts",
  "description": "Get current weather alerts for a US state",
  "parameters": {
    "type": "object",
    "properties": {
      "state": {
        "type": "string",
        "description": "Two-letter US state code (e.g. CA, NY)"
      }
    },
    "required": ["state"]
  },
  "strict": true
}
```

This helper automatically converts your Pydantic model into the proper tool schema, so you don't have to write JSON manually.

In [ ]:
# TODO: Run this code block
from openai.lib._pydantic import to_strict_json_schema
from pydantic import BaseModel

def oai_responses_tool_from_model(name: str, description: str, model: type[BaseModel]):
    return {
        "type": "function",
        "name": name,
        "description": description,
        "parameters": to_strict_json_schema(model),
        "strict": True,
    }

### Step 4 - Set Up the National Weather Service API Wrapper

Before our agent can fetch weather alerts, we need to create helper functions that know how to communicate with the [National Weather Service API](https://www.weather.gov/documentation/services-web-api), specifically for the weather alerts endpoint for a certain state.

These helper functions will be used by our Activity in the next steps.

In [ ]:
# TODO: Run this code block
from typing import Any
import httpx
import json

# Constants for the National Weather Service API
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

def _alerts_url(state: str) -> str:
    """Build the NWS API URL for a given state."""
    return f"{NWS_API_BASE}/alerts/active/area/{state}"

# External calls will happen via activities
async def _make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers, timeout=5.0)
        response.raise_for_status()
        return response.json()

### Step 5 - Build the Tool for the OpenAI Responses API

Now we'll create the Pydantic model that defines what parameters the tool accepts, then use our helper function to convert it into OpenAI's tool definition format.

In [ ]:
# Step 1: In the `description` for state, set it to "Two-letter US state code (e.g. CA, NY)"
# Step 2: Run this code block
from pydantic import BaseModel, Field

class GetWeatherAlertsRequest(BaseModel):
    state: str = Field(description="")  # TODO: Set the description to be "Two-letter US state code (e.g. CA, NY)"

WEATHER_ALERTS_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_weather_alerts", # name 
    "Get weather alerts for a US state.", # description
    GetWeatherAlertsRequest) # model

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "GetWeatherAlertsRequest_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 6 - Implement the Activity

We'll create the Activity function that executes when the LLM decides to use this tool. This function calls the NWS API and returns the weather alert data.

In [ ]:
# Step 1: After the `await`, call the `_make_nws_request` Activity you ran earlier.
# Remember: External calls will happen via Activities.
# Step 2: Run this code block
from temporalio import activity

@activity.defn
async def get_weather_alerts(weather_alerts_request: GetWeatherAlertsRequest) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    data = await ""(_alerts_url(weather_alerts_request.state)) # TODO: Call the `_make_nws_request` Activity you ran earlier
    return json.dumps(data)

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "get_weather_alerts_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 7: Create the Agent

Now we'll create the agent, which is implemented as the **ToolCallingWorkflow**. This Workflow ties everything together:

1. **Makes an initial LLM call** with the user's input and available tools
2. **Checks if the LLM wants to call a tool** by examining the response type
3. **Executes the tool** if requested
4. **Makes a second LLM call** with the tool results to get a final response

In [ ]:
# Step 1: Call the `create` Activity for the initial LLM call
# Step 2: Fill in the `OpenAIResponsesRequest` with `system_instructions`, `input_list`, and `get_weather_alerts.WEATHER_ALERTS_TOOL_OAI`.
# Step 3: Execute the get_weather_alerts activity when a tool is called
# Step 4: Run this code block to load it into the program

from temporalio import workflow
from datetime import timedelta
import json

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class ToolCallingWorkflow:
    @workflow.run
    async def run(self, input: str) -> str:
        input_list = [{"role": "user", "content": input}]
        
        # Initial LLM call with system instructions and tools
        system_instructions = "if no tools seem to be needed, respond in haikus."
        result = await workflow.execute_activity(
            # TODO: Call the `create` Activity here
            OpenAIResponsesRequest(
                model="gpt-4o-mini",
                instructions="",  # TODO: Fill in with system_instructions
                input="",  # TODO: Fill in with input_list
                tools=[]  # TODO: Fill in tools list with get_weather_alerts.WEATHER_ALERTS_TOOL_OAI
            ),
            start_to_close_timeout=timedelta(seconds=30),
        )
        
        # Process the LLM response
        item = result.output[0]
        
        # if the result is a tool call, call the tool
        if item.type == "function_call":
            if item.name == "get_weather_alerts":

                # serialize the output, which is an OpenAI object
                input_list += [
                    i.model_dump() if hasattr(i, "model_dump") else i
                    for i in result.output
                ]
                
                result = await workflow.execute_activity(
                    get_weather_alerts."", # TODO: Execute the get_weather_alerts activity
                    get_weather_alerts.GetWeatherAlertsRequest(state=json.loads(item.arguments)["state"]),
                    start_to_close_timeout=timedelta(seconds=30),
                )
                
                # Add tool call result to input list
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": result
                })
                
                result = await workflow.execute_activity(
                    create,
                    OpenAIResponsesRequest(
                        model="gpt-4o-mini",
                        instructions="return the tool call result in a readable format",
                        input=input_list,
                        tools=[]
                    ),
                    start_to_close_timeout=timedelta(seconds=30),
                )
        
        result = result.output_text
        return result

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "tool_calling_workflow_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Running the Tool-Calling Agent

Now let's run our agent by running our Worker!

In [7]:
# Step 1: Set the Task Queue to be "tool-calling-python-task-queue"
# Step 2: Register `ToolCallingWorkflow` on this Worker
# Step 3: Run this code
import asyncio
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib.pydantic import pydantic_data_converter

async def run_worker() -> None:
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    worker = Worker(
        client,
        task_queue= "", # TODO: Set the Task Queue to be "tool-calling-python-task-queue",
        workflows=[], # TODO: Register `ToolCallingWorkflow` on this Worker
        activities=[
            create,
            get_weather_alerts.get_weather_alerts,
        ],
    )
    
    print(f"Starting the worker....")
    await worker.run()

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "worker_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [8]:
# Run the Worker
worker = asyncio.create_task(run_worker())

### Initiate an Interaction with the Agent

In order to interact with this simple AI agent, we create a Temporal client and execute a workflow.

In [ ]:
# Step 1: Set the task queue of the client to match the task queue the worker is polling
# Step 2: Run this code block
import sys

from temporalio.client import Client
from temporalio.contrib.pydantic import pydantic_data_converter

async def main():
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    query = sys.argv[1] if len(sys.argv) > 1 else "Hello, how are you?"

    # Submit the Tool Calling workflow for execution
    result = await client.execute_workflow(
        ToolCallingWorkflow.run,
        query,
        id="my-workflow-id",
        task_queue="", # TODO: Set the task queue of the client to match the task queue the worker is polling
    )
    print(f"Result: {result}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "client_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

NOTICE...? WEBUI...? FOLLOWUP QUESTIONS...?

The key capability is that the LLM decides autonomously whether to use the weather
   alerts tool or not. You don't hardcode "if user mentions a state, call the API" -
   instead, the LLM evaluates the input and decides if calling get_weather_alerts
  will help achieve the goal.

### What is an Agentic Loop?

At a high level, AI agents have an **event loop**, and in that event loop, we make calls out to the LLM to ask it for directions. **The LLM is what drives the flow** of the application.

We might invoke some downstream actions (e.g., make microservice requests, query databases, call external APIs). Then we may consult with the user, and then go through the loop again.

In the loop, it:
- **Plans**: The LLM analyzes the current state and decides what to do next
- **Executes**: The agent calls relevant tools or APIs
- **Observes**: Reviews results and updates context
- **Repeats** until the LLM hits its goal or the user stops it

<img src="https://i.postimg.cc/MGPGgMH1/the-agentic-loop.png" width="300"/>

Imagine a travel agent. The agent uses tools to search for flights, collect travel details, and then handle bookings. The loop allows the agent to use the output from one tool (e.g., flight search results) to inform the next step, like using flight details to book the trip or charging the user.

The agent is generally implemented as an event loop that is kicked off with an expression of some goal, such as "Book a round-trip flight from New York to Paris for June 15-22 under $800."

In the loop, it:
- **Asks the LLM to determine the next steps in the flow**: The LLM analyzes the current state, available info, and decides what to do next like search for flights, gather user preferences about departure time
- **Invokes one or more tools to perform those actions**: The agent calls relevant APIs or tools like flight search engines or payment processors
- **Evaluates results and continues**: The agent assesses if the goal has been achieved or if more steps are needed.

### Why is the Agentic Loop Powerful?

- The AI makes its own decisions
- Can adapt to different situations dynamically
- Can use different tools in different orders based on context

### The Agentic Loop in Temporal

- In Temporal, an agentic loop is represented as a **Workflow**. 
- We use Activities for LLMs and Tool Calls for automatic retries and durability
- Complete step tracking
    - If you trigger an LLM invocation, then execute a tool, and finally hand off control to another agent, Temporal records every step of that process in its event history.
- Durable Execution
    - _Automatic recovery_: If something goes wrong—a crash, network glitch, or service interruption—Temporal automatically replays the Workflow from the beginning, using the recorded event history to skip already-completed steps.
    - _No data loss_: Temporal persistently records application progress and stores all Activity results in its durable event log.
    - _No duplicate work_: Once an Activity completes successfully, its result is recorded. On replay, Temporal returns the stored result instead of re-executing the Activity.

### Thank you for attending this workshop. Feedback?

Please leave your feedback for this workshop [here](https://docs.google.com/forms/d/e/1FAIpQLSfkHMev6KCNGFHpVNydyjgAh2ALeHNVYv9TaSrAoBsT0KmNHQ/viewform?usp=header)!

### What's Next?

This workshop introduced you to the foundations of AI, human in the loop, agentic loops, and why durability matters.

### Resources

- [Temporal Docs](https://docs.temporal.io/)
- [Code Repository](https://github.com/temporalio/edu-ai-workshop-agentic-loop)
- [Free Temporal Courses](https://learn.temporal.io/courses/)
- Join our [Community Slack](https://t.mp/slack)